In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from scipy import stats
from sklearn.model_selection import KFold

from kerashypetune import KerasRandomSearch,KerasRandomSearchCV

In [2]:
def set_seed_TF2(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

In [4]:
def get_model(param):
        
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': stats.randint(32, 128),
    'lr': stats.uniform(1e-4, 0.1), 
    'epochs': 100, 
    'batch_size': 512
}

# Fixed validation search

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model  
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

krs = KerasRandomSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
krs.set_seed(set_seed_TF2, seed=33)
krs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'unit_1': 64, 'unit_2': 61, 'lr': 0.01444716297030787, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
SCORE: 0.95208 at epoch 18

***** (2/15) *****
Search({'unit_1': 128, 'unit_2': 105, 'lr': 0.07599530465963981, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00047: early stopping
SCORE: 0.87614 at epoch 37

***** (3/15) *****
Search({'unit_1': 128, 'unit_2': 81, 'lr': 0.06027688797008003, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
SCORE: 0.89166 at epoch 10

***** (4/15) *****
Search({'unit_1': 64, 'unit_2': 96, 'lr': 0.0077648438758605946, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
SCORE: 0.94971 at epoch 18

***

In [7]:
krs.scores

[0.95208,
 0.87614,
 0.89166,
 0.94971,
 0.95241,
 0.92778,
 0.88289,
 0.88761,
 0.94566,
 0.85083,
 0.89976,
 0.94836,
 0.90753,
 0.92913,
 0.89436]

In [8]:
krs.best_score

0.95241

In [9]:
krs.best_params

{'unit_1': 128,
 'unit_2': 125,
 'lr': 0.016525580116352412,
 'epochs': 11,
 'batch_size': 512,
 'steps_per_epoch': 12}

In [10]:
krs.best_model

# Fixed validation search Generator

In [11]:
gen = ImageDataGenerator(
         rotation_range=90,
         width_shift_range=0.1,
         height_shift_range=0.1,
         zoom_range=0.2)

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

krs = KerasRandomSearch(hypermodel, param_grid, n_iter=6, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
krs.set_seed(set_seed_TF2, seed=33)
krs.search(gen.flow(np.expand_dims(x_train,-1), y_train, batch_size=param_grid['batch_size'], seed=33), 
           validation_data=(np.expand_dims(x_test,-1), y_test), 
           callbacks=[es], steps_per_epoch=len(x_train)//param_grid['batch_size'])


6 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/6) *****
Search({'unit_1': 64, 'unit_2': 67, 'lr': 0.029108453941641077, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
SCORE: 0.65913 at epoch 11

***** (2/6) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.051806429397159084, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
SCORE: 0.49713 at epoch 16

***** (3/6) *****
Search({'unit_1': 128, 'unit_2': 69, 'lr': 0.06736172401499606, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
SCORE: 0.48127 at epoch 10

***** (4/6) *****
Search({'unit_1': 64, 'unit_2': 33, 'lr': 0.09865537618503832, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
SCORE: 0.31657 at epoch 10

***** (5/

In [13]:
param_grid

{'unit_1': [128, 64],
 'unit_2': <scipy.stats._distn_infrastructure.rv_frozen at 0x195b0871f88>,
 'lr': <scipy.stats._distn_infrastructure.rv_frozen at 0x195b0871dc8>,
 'epochs': 100,
 'batch_size': 512}

In [14]:
krs.scores

[0.65913, 0.49713, 0.48127, 0.31657, 0.75329, 0.41006]

In [15]:
krs.best_score

0.75329

In [16]:
krs.best_params

{'unit_1': 128,
 'unit_2': 79,
 'lr': 0.03890596911515373,
 'epochs': 20,
 'batch_size': 512,
 'steps_per_epoch': 11}

In [17]:
krs.best_model

# Cross validation search

In [18]:
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

X.shape, y.shape

((8999, 28, 28), (8999,))

In [19]:
cv = KFold(n_splits=3, random_state=33, shuffle=True)

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)

krs = KerasRandomSearchCV(hypermodel, param_grid, cv=cv, n_iter=5, sampling_seed=33,
                          monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
krs.set_seed(set_seed_TF2, seed=33)
krs.search(X, y, callbacks=[es])


##################
###  Fold 001  ###
##################

5 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/5) *****
Search({'unit_1': 64, 'unit_2': 97, 'lr': 0.024872053525859684, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
SCORE: 0.93833 at epoch 5

***** (2/5) *****
Search({'unit_1': 128, 'unit_2': 77, 'lr': 0.04678478968770971, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
SCORE: 0.93033 at epoch 19

***** (3/5) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.02430847222686987, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
SCORE: 0.94467 at epoch 17

***** (4/5) *****
Search({'unit_1': 64, 'unit_2': 114, 'lr': 0.02425552582635281, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00

In [20]:
krs.folds_scores

{'fold 1': [0.93833, 0.93033, 0.94467, 0.94133, 0.892],
 'fold 2': [0.94, 0.92633, 0.94933, 0.94667, 0.83533],
 'fold 3': [0.94231, 0.92531, 0.94998, 0.93765, 0.88196]}

In [21]:
krs.folds_best_score

{'fold 1': 0.94467, 'fold 2': 0.94933, 'fold 3': 0.94998}

In [22]:
krs.folds_best_params

{'fold 1': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 17,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 2': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 11,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 3': {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 19,
  'batch_size': 512,
  'steps_per_epoch': 12}}

In [23]:
krs.folds_best_models

{'fold 1': <tensorflow.python.keras.engine.sequential.Sequential at 0x195b68d7548>,
 'fold 2': <tensorflow.python.keras.engine.sequential.Sequential at 0x195b6e12c88>,
 'fold 3': <tensorflow.python.keras.engine.sequential.Sequential at 0x195b25ed1c8>}

In [24]:
krs.best_params_score

0.94799

In [25]:
krs.best_params

[{'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 17,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 11,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 37,
  'lr': 0.02430847222686987,
  'epochs': 19,
  'batch_size': 512,
  'steps_per_epoch': 12}]